In [31]:
import re
import string
import pickle
import numpy as np
import pandas as pd
from nltk.stem import PorterStemmer

ps = PorterStemmer()

#### Data preprocessing

In [32]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [33]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [34]:
def preprocess_data (text):
    data = pd.DataFrame([text], columns=['tweet']) 

    data['tweet'] = data['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data['tweet']= data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data['tweet'] = data['tweet'].apply(remove_punctuations)
    data['tweet'] = data['tweet'].str.replace('\d+', '', regex=True)
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))

    return data['tweet']

In [ ]:
text = 'great video! I like it!'

preprocessed_text = preprocess_data(text)

print(preprocessed_text)

0    great video like much
Name: tweet, dtype: object


#### vocab building

In [36]:
vocab = pd.read_csv('../static/model/vocabulary.txt', header=None)
tokens = vocab[0].tolist() 

#### vectorization

In [37]:
def vectorizer(ds, vocabulary):
    vectorized_list = []

    for sentence in ds:
        sentence_list = np.zeros(len(vocabulary))

        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_list[i] = 1

        vectorized_list.append(sentence_list)  # Only append after processing the sentence

    vectorized_list_new = np.asarray(vectorized_list, dtype=np.float32)
    return vectorized_list_new

In [38]:
vectorized_text = vectorizer(preprocessed_text, tokens)

#### analyze

In [39]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [40]:
model.predict(vectorized_text)

array([0])